In [74]:
from ultralytics import YOLO

In [75]:
import cv2

In [76]:
image = cv2.imread("./documents/train/images/image_380.png")

In [99]:
doc_type_detect_model = YOLO("./runs/detect/train8/weights/best.pt")
series_number_detect_model = YOLO("./runs/detect/train10/weights/best.pt")

result_to_server={'confidence': None, 'doc_type': None}

In [78]:
from utils import crop_image, get_grayscale, thresholding, result_postprocess

In [79]:
doc_type_pred = doc_type_detect_model.predict(source=image)


0: 640x480 1 pts, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [97]:
confidence = doc_type_pred[0].boxes.conf[0].cpu().item()
doc_type = doc_type_pred[0].names[int(doc_type_pred[0].boxes.cls[0].cpu().item())]

result_to_server['confidence'] = confidence
result_to_server['doc_type'] = doc_type

In [81]:
xyxy_coordinates = doc_type_pred[0].boxes.xyxy[0].cpu().numpy().astype(int) 
x1, y1, x2, y2 = xyxy_coordinates  

cropped_image = crop_image(image, x1, y1, x2, y2)

In [82]:
series_numbers_pred = series_number_detect_model.predict(source=cropped_image)


0: 640x480 1 number, 1 series, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [89]:
xyxy_coordinates = series_numbers_pred[0].boxes.xyxy.cpu().numpy().astype(int) 
labels = [series_numbers_pred[0].names[item] for item in series_numbers_pred[0].boxes.cls.cpu().numpy()]

In [86]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [90]:
i = 0

ocr_dict = {'number': [], 'series': []}

for item in xyxy_coordinates:
    x1, y1, x2, y2 = item
    cropped_image_tes = crop_image(cropped_image, x1, y1, x2, y2)

    gray = get_grayscale(cropped_image_tes)
    thresh = thresholding(gray)

    ocr_result = pytesseract.image_to_string(cropped_image_tes, lang='rus', config='--psm 13 --oem 3')

    final_res = result_postprocess(ocr_result)

    if labels[i] == 'number':
        ocr_dict['number'].append(final_res)
    if labels[i] == 'series':
        ocr_dict['series'].append(final_res)

    i += 1


In [91]:
ocr_dict

{'number': ['587970'], 'series': ['52НУ']}

In [101]:
result_to_server['series_numbers'] = ocr_dict

In [102]:
result_to_server

{'confidence': None,
 'doc_type': None,
 'series_numbers': {'number': ['587970'], 'series': ['52НУ']}}